In [1]:
from math import lgamma
from numba import jit
import numpy as np
from scipy.stats import beta

# Описание работы Байесовских алгоритмов

## Принцип работы
Предполагается некое априорное распределения метрик (биномиальные, нормальное,
экспоненциальное и т.д.). Затем, используя полученные данные (новую информацию), мы
пересчитываем распределения для метрик. При помощи распределения

## Основные критерии принятия решений

Основная проблема Байесовских тестов по сравнению с классическими - проблема
контроля ошибки 1 и 2 рода. Особенно это проявляется в случае ранней остановки
алгоритма (когда мы заканчиваем эксперимент при достижении определенных значений
критериев). Ниже будут описаны основные критерии принятия решений
в случае байесовских тестов
(как для алгоритма Томпсона, так и для разбивки 50 на 50).

### Вероятность превосходства для конверсии
Английские термины: *chance to beat control*, *probability superiority*

В случае алгоритма на конверсию у нас в качестве априорного распределения берется
биномиальное. Для того чтобы пересчитать апостериорную вероятность для биномиального
распределения, используется сопряженное ему бета-распределение.
Вероятность превосходства  можно посчитать двумя способами: по точной формуле или при помощи генерации
случайных величин из бета-распределения с параметрами на момент пересчета.

Точная формула описана [здесь](https://www.johndcook.com/UTMDABTR-005-05.pdf).

Метод сэмплирования следующий:

1. Сэмплируем значения из бета-распределений с параметрами
на текущий момент (желательно как можно больше значений)
2. Вычисляем разницу между конверсиями для каждого варианта из
этих распределений
3. Смотрим сумму положительных значений для теста
4. Суммируем и делим на число сэмплирований

In [3]:
from src.bayesian import *
# Посмотрим на результаты, полученные разными способами
alphas, betas = [10, 20], [100, 200]
print(f"Точная формула: {calc_prob_between(alphas, betas)}")
print(f"Сэмплирование: {chance_to_beat_all(np.array(alphas), np.array(betas))}")

Точная формула: 0.5148327134706592
Сэмплирование: [0.485115 0.51511 ]


Отметим, что сэмплирование приводит примерно к тому же результату, но при большом
количестве генераций случайных распределений (например, 1 млн).

Важный момент, который важно понять - эта вероятность превосходства
верна только для текущего набора наблюдений. Другими словами,
мы не знаем, как все изменится при добавлении других данных.
А вот как раз в случае p-value мы можем предположить, в скольких
случаях выигрышный на данный момент вариант может на самом
деле оказаться проигрышным в будущем.

Экспериментальным путем однако выявлено, что в случае
Байесовских тестов без остановки для контроля ошибок
1,2 рода, вероятности превосходства вполне достаточно.

### Ожидаемые потери
Английский термин: *expected losses*

Данный критерий описан в [статье](https://www.chrisstucchio.com/pubs/slides/gilt_bayesian_ab_2015/slides.html#1)
*Chris Stucchio* - разработчика
самого лучшего инструмента для АБ-тестов
*Visual Website Optimizer*.


Ожидаемые потери контрольного / тестового варианта - сколько мы ожидаем в среднем потерь в случае,
если выберем контрольный / тестовый вариант. Считается похожим
образом, как и сэплированная вероятность превосходства:
1. Сэмплируем значения из бета-распределений с параметрами
на текущий момент (желательно как можно больше значений)
2. Вычисляем разницу между конверсиями для каждого варианта из
этих распределений
3. Отрицательной разнице присваиваем 0
4. Суммируем и делим на число сэмплирований

Определяем некий порог и в бой!

### Размер эффекта
Английский термин: *effect size*

Размер эффект дополняет показатель разницы конверсий и
рассчитывается по такой формуле:

$
\frac{\hat{p_1} - \hat{p_2}} {
[n_1 * \hat{p_1} * (1 - \hat{p_1}) * (n_1 - 1) + n_2 * \hat{p_2} *
(1 - \hat{p_2}) * (n_2 - 1)] /
(n_1 + n_2 - 2)}
$

### Достоверные интервалы
Английский термин: *credible intervals*

Достоверные интервалы являются аналогом доверительных интервалов в случае
классических АБ-тестов. Однако, достоверные интервалы фиксируют
нашу текущую неопределенность о потенциальных значениях параметра и
могут быть интерпретированы как вероятностное утверждение о параметре.

Доверительные интервалы отражают неопределенность относительно
полученного нами интервала (т.е. содержит ли он истинное значение или нет).
Таким образом, их нельзя интерпретировать как вероятностное утверждение
об истинных значениях параметров.

Сравним интерпретацию 95% доверительного и достоверного интервала.

В случае доверительного интервала в 95% экспериментах истинное значение
параметра будет лежать в данном диапазоне.

В случае достоверного интервала 95% ВСЕХ ЗНАЧЕНИЙ лежит в этом диапазоне.

### Область практической эквивалентности
Английский термин: *ROPE - region of practical equivalence*

Данный критерий может помочь принять нулевую гипотезу - случай
отсутствия разницы между разными вариантами теста. Рекомендуемое значение
границ *ROPE* = +- 0.1 для размера эффекта.

Если *credible intervals* полностью лежат внутри ROPE, то
делаем вывод, что разницы нет.

## Пример реализации Байесовских тестов с разными критериями

Будем пользоваться пакетом abyes и собственными вычислениями
